In [8]:
import time,os, pickle
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torchsummary import summary
from PIL import Image
import random

class Task():
    def __init__(self, tid=-1, arrive_time = -1, start_time = -1, finish_time = -1):
        # self.model_name = model_name
        self.tid = tid
        self.arrive_time = arrive_time
        self.start_time = start_time
        self.finish_time = finish_time
        self.infer_time = -1
        self.queue_time = -1

    def finalize(self, if_print=True):
        self.infer_time = self.finish_time - self.start_time
        self.queue_time = self.start_time - self.arrive_time
        if if_print: print('infer time: ', self.infer_time, 'queue time: ', self.queue_time)


In [18]:
with open('./homomodel_result/results_0.2_4.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.010312288343083014
[0.24668952100910246, 0.010564533004071563, 0.009506347007118165, 0.009067004022654146, 0.27897331898566335, 0.2836584930191748, 0.009590536006726325, 0.2879717120085843, 0.011037599993869662, 0.011280801030807197, 0.009509883006103337, 0.011175290972460061, 0.012890344019979239, 0.011582384002394974, 0.009461281995754689, 0.011142347997520119, 0.01166563096921891, 0.011855993012432009, 0.0114149060100317, 0.010380183986853808, 0.013091504981275648, 0.012441011029295623, 0.01175718504237011, 0.009073085035197437, 0.01095934200566262, 0.011384868994355202, 0.011832026997581124, 0.008547267992980778, 0.009937082999385893, 0.010522594035137445, 0.014789575012400746, 0.010243925033137202, 0.010238193965051323, 0.010892043996136636, 0.014914150990080088, 0.009750019002240151, 0.00870556803420186, 0.009997347020544112, 0.013887644978240132, 0.009830582013819367, 0.009954947978258133, 0.010132372961379588, 0.01103251101449132, 0.010945736023131758, 0.013550164992921054, 0

In [19]:
with open('./homomodel_result/results_0.3_4.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.010922661792738375
[0.2705634049489163, 0.2695476679946296, 0.2704760089982301, 0.011263818945735693, 0.010847769968677312, 0.2853370700031519, 0.016958979016635567, 0.010544756019953638, 0.013135218003299087, 0.012211556022521108, 0.011281191953457892, 0.010341390036046505, 0.012517095950897783, 0.012650308955926448, 0.01168627996230498, 0.011089358013123274, 0.011139563983306289, 0.011164119991008192, 0.011531807016581297, 0.010790721978992224, 0.013227051997091621, 0.012788070016540587, 0.009986596996895969, 0.010595893021672964, 0.010622924019116908, 0.012595615000464022, 0.012575436965562403, 0.011728350014891475, 0.011448609991930425, 0.011486010975204408, 0.011549901973921806, 0.010092458047438413, 0.010845215001609176, 0.011482444009743631, 0.01210175798041746, 0.010215891990810633, 0.010351570032071322, 0.010815869958605617, 0.0145261759753339, 0.012275327986571938, 0.011620205012150109, 0.011053288995753974, 0.01143429201329127, 0.010411333001684397, 0.010695722012314945, 0

In [20]:
with open('./homomodel_result/results_0.4_20.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.057116101087432966
[0.3698880310403183, 0.011805486981756985, 0.01061006000963971, 0.01081920595606789, 0.4178761799703352, 0.42149112501647323, 0.013613956980407238, 0.40783142199506983, 0.014550683030392975, 0.022156075050588697, 0.018386305950116366, 0.019567415001802146, 0.021043254993855953, 0.4437776970444247, 0.024151468998752534, 0.02510849299142137, 0.026973870990332216, 0.025371032010298222, 0.02434061700478196, 0.023039981024339795, 0.023364196007605642, 0.022559089004062116, 0.5020172080257908, 0.5030068440246396, 0.02111478999722749, 0.021035669022239745, 0.020960027002729475, 0.0247093359939754, 0.02442343399161473, 0.49902313901111484, 0.025007651944179088, 0.02623887703521177, 0.02631465101148933, 0.02594262600177899, 0.5117203170084395, 0.02759543899446726, 0.5423081110347994, 0.026745716982986778, 0.5319437980069779, 0.02753355202730745, 0.02967957203509286, 0.028822145017329603, 0.031012109015136957, 0.031344728951808065, 0.5590657689608634, 0.5554793880437501, 0.5

In [21]:
with open('./homomodel_result/results_0.4_4.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.0112990822993397
[0.26013703702483326, 0.26337313395924866, 0.009935320995282382, 0.010958199971355498, 0.00990048301173374, 0.2836664660135284, 0.010639394982717931, 0.2905694550136104, 0.010352270968724042, 0.012081961031071842, 0.010254174994770437, 0.011621939018368721, 0.01132721995236352, 0.010260997980367392, 0.013740294962190092, 0.012077883002348244, 0.012967210030183196, 0.009605013998225331, 0.01266526704421267, 0.012562010961119086, 0.0121380869532004, 0.011051526991650462, 0.011279689031653106, 0.011088367027696222, 0.011367426021024585, 0.011887311993632466, 0.011946304002776742, 0.011241117026656866, 0.01107613299973309, 0.010851237981114537, 0.01116418099263683, 0.01246442599222064, 0.012794833048246801, 0.012523618002887815, 0.011844470049254596, 0.010077390004880726, 0.010105552966706455, 0.010691685019992292, 0.011154522013384849, 0.012338207045104355, 0.01201760902768001, 0.01142505498137325, 0.011349822976626456, 0.0103140989667736, 0.010626772011164576, 0.011172

In [23]:
with open('./homomodel_result/results_0.4_8.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.02271910913091624
[0.25088272796710953, 0.26303845801157877, 0.011205739981960505, 0.27100962499389425, 0.011415518005378544, 0.011285019980277866, 0.2732544420287013, 0.009847453038673848, 0.011113173968624324, 0.01229052699636668, 0.014157857978716493, 0.296804168028757, 0.011586141015868634, 0.011616558011155576, 0.011698063055519015, 0.011913430993445218, 0.3064477859879844, 0.01691682997625321, 0.016419096034951508, 0.015741059032734483, 0.015014150994829834, 0.01584101002663374, 0.3110428600339219, 0.3199172879685648, 0.0168027930194512, 0.019169450970366597, 0.019122602010611445, 0.021173692017327994, 0.021220711001660675, 0.021852129022590816, 0.022322439996059984, 0.02333615202223882, 0.022836404968984425, 0.022344963042996824, 0.022288576001301408, 0.022307050006929785, 0.022352195985149592, 0.0223008279572241, 0.022353608976118267, 0.022288135020062327, 0.022298444993793964, 0.02234027296071872, 0.022340474009979516, 0.02234029397368431, 0.022316449030768126, 0.02234041399

In [24]:
with open('./homomodel_result/results_0.4_12.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.034035066398350834
[0.2710588280460797, 0.011191261990461498, 0.28052916598971933, 0.29310287901898846, 0.010714076983276755, 0.2889972340199165, 0.29475597402779385, 0.009470087999943644, 0.013633752998430282, 0.30226001399569213, 0.01806937495712191, 0.29864336602622643, 0.01827496400801465, 0.3106607749941759, 0.3116842560120858, 0.015613527037203312, 0.022101521026343107, 0.3271387520362623, 0.02337735000764951, 0.023667590983677655, 0.023872960009612143, 0.02430690301116556, 0.024532381037715822, 0.02515405899612233, 0.025645059999078512, 0.02784393000183627, 0.33849981595994905, 0.0284718910115771, 0.029000444046687335, 0.030558787984773517, 0.35641333501553163, 0.03040133003378287, 0.030416678986512125, 0.030400408024434, 0.030480429995805025, 0.03027111303526908, 0.030438760994002223, 0.030426056997384876, 0.030914452974684536, 0.03315649495925754, 0.03312239900697023, 0.03312496398575604, 0.033601518953219056, 0.03363172599347308, 0.03356649196939543, 0.033573444990906864, 0

In [25]:
with open('./homomodel_result/results_0.4_16.pkl', 'rb') as file: 
    results = pickle.load(file)
infer_times = [r.infer_time for r in results.values()]
print(np.sum(infer_times)/len(infer_times))
print(infer_times)

0.04523891569307394
[0.26719991298159584, 0.01263716397807002, 0.29248318797908723, 0.014438348996918648, 0.01364491495769471, 0.011089058010838926, 0.010256570007186383, 0.31218941701808944, 0.31945863098371774, 0.011191562982276082, 0.011944539961405098, 0.33154733502306044, 0.014382242981810123, 0.328722017002292, 0.01590429898351431, 0.3446683359798044, 0.025250361999496818, 0.35609143803594634, 0.023724097991362214, 0.024178570020012558, 0.023742894001770765, 0.02602595300413668, 0.02590757899452001, 0.35098190896678716, 0.35277598100947216, 0.34548126900335774, 0.02655383397359401, 0.02790682000340894, 0.03140127600636333, 0.03154740302124992, 0.031652834033593535, 0.03100697899935767, 0.03129484399687499, 0.031374435988254845, 0.03159105603117496, 0.032101033022627234, 0.03384566103341058, 0.03314565395703539, 0.03304327995283529, 0.4223600529949181, 0.0334956279839389, 0.03336816601222381, 0.03343442198820412, 0.03339473600499332, 0.42055575200356543, 0.033498834003694355, 0.03

In [26]:
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) 
model.load_state_dict(torch.load('./weights/resnet50_cifar10.pth'))
for i, (name, param) in enumerate(model.named_parameters()):
    print(i, name, param.requires_grad)

FileNotFoundError: [Errno 2] No such file or directory: './weights/resnet50_cifar10.pth'

In [10]:
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.last_channel, 10)

for i, module in enumerate(list(model.modules())):
    print(i,module)

0 MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, ep

In [11]:
# for i, child in enumerate(list(model.children())[:-1]):
#     print(i,child)
for i, child in enumerate(list(model.children())):
    for j, sub_child in enumerate(list(child.children())):
        print(i,j,sub_child)

0 0 Conv2dNormActivation(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace=True)
)
0 1 InvertedResidual(
  (conv): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
0 2 InvertedResidual(
  (conv): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): Conv2dNormActivation

In [3]:
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.last_channel, 10)
detail_layers = []
for i, child in enumerate(list(model.children())):
    for j, sub_child in enumerate(list(child.children())):
        detail_layers.append(sub_child)
print(len(detail_layers))

for i, layer in enumerate(detail_layers):
    print(i, layer)

21
0 Conv2dNormActivation(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace=True)
)
1 InvertedResidual(
  (conv): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
2 InvertedResidual(
  (conv): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): Conv2dNormActivation(
 

In [14]:
test = [0,1,2,3,4,5]
test[-1:]

[5]

In [12]:
class EarlyExitMobileNet(nn.Module):
    def __init__(self, original_model, exit_layer = -5, num_classes=10):
        super(EarlyExitMobileNet, self).__init__()
        # expand MobileNet
        detail_layers = []
        for i, child in enumerate(list(original_model.children())):
            for j, sub_child in enumerate(list(child.children())):
                detail_layers.append(sub_child)

        self.features = nn.Sequential(*detail_layers[:exit_layer])
        exit_channel = detail_layers[exit_layer-1].out_channels
        print(f'exit_channel: {exit_channel}')
        print(self.features)
        # Freeze the features part
        for param in self.features.parameters():
            param.requires_grad = False
        
        # Early exit branch
        # self.exit = nn.Sequential(*detail_layers[exit_layer:])
        self.early_exit = nn.Sequential(
            nn.Conv2d(exit_channel, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False),  # Conv layer with 512 output channels
            nn.BatchNorm2d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),                            # BatchNorm layer
            nn.ReLU6(inplace=True),
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(in_features=1280, out_features=num_classes, bias=True)
        )

    def forward(self, x):
        x = self.features(x)
        # Early exit logic
        early_exit_output = self.early_exit(x)
        
        # Continue with the remaining layers
        # main_exit_output = self.main_exit(x)
        
        return early_exit_output# , main_exit_output

# 定义新的模型
num_classes = 10  # 根据你的任务更改类别数量
exit_layer = -3
ee_model = EarlyExitMobileNet(model, exit_layer, num_classes)

# print(type(ee_model))
# for i, child in enumerate(list(ee_model.children())):
#     for j, sub_child in enumerate(list(child.children())):
#         print(i,j,sub_child)

exit_channel: 320
Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_sta